In [33]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
from langchain_community.tools import tool
from langchain.agents import create_react_agent, AgentExecutor
from langchain.memory import ConversationSummaryMemory
from langchain.prompts import PromptTemplate
from langchain.utilities import SerpAPIWrapper
from langchain.document_loaders import DirectoryLoader, PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from sentence_transformers import SentenceTransformer

load_dotenv()
groq_api_key = os.getenv("GROQ_API_KEY")

llm = ChatGroq(
    groq_api_key=groq_api_key,
    model_name="llama-3.1-8b-instant",
    temperature=0.7
)

In [34]:
#To load pdf files

pdf_folder_path = "./Data/RagFiles"

loader = DirectoryLoader(
    pdf_folder_path,
    glob="*.pdf",
    loader_cls=PyPDFLoader
)
raw_docs = loader.load()

print(f"Loaded {len(raw_docs)} PDF documents from {pdf_folder_path}")


Loaded 3 PDF documents from ./Data/RagFiles


In [35]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200
)
docs = splitter.split_documents(raw_docs)
print(f"After splitting, we have {len(docs)} chunks total.")


After splitting, we have 8 chunks total.


In [ ]:
class SentenceTransformerEmbeddings:
    def __init__(self, model_name='all-MiniLM-L6-v2'):
        self.model = SentenceTransformer(model_name)
    
    def embed_documents(self, texts):
        return self.model.encode(texts).tolist()
    
    def embed_query(self, text):
        return self.model.encode([text])[0].tolist()

embedding_func = SentenceTransformerEmbeddings()

db = Chroma.from_documents(docs, embedding=embedding_func, persist_directory="./chroma_db")

retriever = db.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [36]:
@tool
def add(input_str: str) -> str:
    """Add two numbers from a string like 1, 2."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a + b)

@tool
def subtract(input_str: str) -> str:
    """Subtract two numbers from a string like 5, 3."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a - b)

@tool
def multiply(input_str: str) -> str:
    """Multiply two numbers from a string like 2, 3."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a * b)

@tool
def divide(input_str: str) -> str:
    """Divide two numbers from a string like 6, 2."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    if b == 0:
        return "Error: Division by zero"
    return str(a / b)

@tool
def power(input_str: str) -> str:
    """Raise one number to another from a string like 2, 3."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a ** b)

@tool
def modulus(input_str: str) -> str:
    """Compute the remainder for 'a mod b' from a string like 5, 3."""
    a_str, b_str = input_str.split(",")
    a = float(a_str.strip())
    b = float(b_str.strip())
    return str(a % b)
@tool
def web_search(query: str) -> str:
    """
    Search the web using SerpAPI for the given query.
    Returns a summary of the top results.
    """
    try:
        serpapi = SerpAPIWrapper()
        return serpapi.run(query)
    except Exception as e:
        return f"Error searching: {str(e)}"

@tool
def pdf_rag(query: str) -> str:
    """
    Retrieve relevant PDF content based on the query.
    
    Args:
        query (str): Search query to find relevant content
    
    Returns:
        str: Chunks of relevant content
    """

    results = retriever.get_relevant_documents(query)
    
    if not results:
        return f"No relevant documents found for query: {query}"

    return results

@tool
def summarize_file(query: str) -> str:
    """
    Summarizes files related to the given query.
    
    Args:
        query (str): Search query to find relevant files
    
    Returns:
        str: Comprehensive summary of matching files
    """
    all_docs = docs
    chunk_summary=[]

    for doc in all_docs:
        chunk_prompt = f"""
        Provide a summary for the following document content
        related to {query}. Present the summary in detail.

        Document Excerpt:
        {doc.page_content[:5000]}

        Detailed Summary:
        """
        try:
            note = llm.invoke(chunk_prompt).content
            chunk_summary.append(note)
        except Exception as e:
            chunk_summary.append(f"Error generating notes for a chunk: {str(e)}")

    combined_notes_prompt = f"""
    The following are summaries from different document chunks related to {query}. 
    Summarize them into a single structured set.

    Individual Summaries:
    {"\n\n".join(chunk_summary)}

    Final Summary:
    """

    try:
        final_summary = llm.invoke(combined_notes_prompt).content
        return final_summary
    except Exception as e:
        return f"Error generating final notes: {str(e)}"
    
@tool
def notes_file(query: str) -> str:
    """
    Makes notes of the files related to the given query.
    
    Args:
        query (str): Search query to find relevant files
    
    Returns:
        str: Notes from matching files
    """
    all_docs = docs
    chunk_notes = []

    for doc in all_docs:
        chunk_prompt = f"""
        Provide a concise set of structured notes for the following document content
        related to {query}. Present the notes in clear bullet points.

        Document Excerpt:
        {doc.page_content[:5000]}

        Bullet-Point Notes:
        """
        try:
            note = llm.invoke(chunk_prompt).content
            chunk_notes.append(note)
        except Exception as e:
            chunk_notes.append(f"Error generating notes for a chunk: {str(e)}")

    combined_notes_prompt = f"""
    The following are notes from different document chunks related to {query}. 
    Summarize them into a single structured set of notes with key takeaways.

    Individual Notes:
    {"\n\n".join(chunk_notes)}

    Final Notes:
    """

    try:
        final_notes = llm.invoke(combined_notes_prompt).content
        return final_notes
    except Exception as e:
        return f"Error generating final notes: {str(e)}"

    
tools = [add, subtract, multiply, divide, power, modulus, web_search, pdf_rag, summarize_file, notes_file]

In [44]:
memory = ConversationSummaryMemory(
    llm=llm,
    output_key="output"
)

prompt_template = PromptTemplate(
    template="""
You are an AI agent with access to these tools:
{tool_names}

Tools:
{tools}

When you need to use a tool, respond exactly in this format:

Thoughts: [Reasoning]
Action: [tool name]
Action Input:[tool input]

"pdf_rag, summarize_file, notes_file take priority over web_search."
"Use web_search only if any of the present tools are unable to provide a relevant answer."
“If you take an action, do not produce a final answer. If you produce a final answer, do not produce an action.”

Thoughts: [Reasoning]

Final Answer: [answer here]

User Question: {user_input}

{agent_scratchpad}
""",
    input_variables=["tool_names", "tools", "user_input", "agent_scratchpad"]
)



agent = create_react_agent(
    llm=llm,
    tools=tools,
    prompt=prompt_template.partial(
        tool_names=", ".join([t.name for t in tools]),
        tools="\n".join([f"{t.name}: {t.description}" for t in tools])
    )
)

agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    memory=memory,
    verbose=True,
    return_intermediate_steps=True,
    handle_parsing_errors=True
)

In [51]:
response = agent_executor.invoke({"user_input": "can you do  1.346 to the power 7.289"})
print(response)



> Entering new AgentExecutor chain...
Thoughts: The user asked to raise 1.346 to the power 7.289. I need to find a way to perform this operation.

Action: power
Action Input: 1.346, 7.2898.721866632573068Thoughts: The operation was performed successfully. I will now provide the result of the power operation.

Final Answer: 8.722

> Finished chain.
{'user_input': 'can you do  1.346 to the power 7.289', 'history': 'Current summary: \nThe human asks the AI to do addition between 1.346 and 7.289, to which the AI responds with the result of 8.635.\n\nNew lines of conversation:\nHuman: can you do multiplication between 1.346 and 7.289\nAI: 9.810994\n\nNew summary: \nThe human asks the AI to perform mathematical operations, starting with addition between 1.346 and 7.289, resulting in 8.635, and then proceeds to ask for multiplication between the same numbers, receiving a result of 9.810994.', 'output': '8.722', 'intermediate_steps': [(AgentAction(tool='power', tool_input='1.346, 7.289', log